In [ ]:
import tensorflow as tf
from tensorflow_probability import distributions as tfd
import tensorflow_probability as tfp
import numpy as np
import matplotlib.pyplot as plt

import helper

In [ ]:
import matplotlib.gridspec as gridspec

def plot_likelihoods(data, sigma_reward_model, N=500, t_step=2):
    gamma_eval_points = tfd.Uniform([-.2,-.2],[1.2,1.2]).sample(N)
    subplots = [plt.subplot(grid_loc) for grid_loc in gridspec.GridSpec(1, 8)]
    for i,sp in enumerate(subplots):
        t=t_step*i+1
        llhs = np.array([helper.model_llh(data['z'][:t], data['r'][:t], gamma=gamma_eval_points[j], sigma_reward=sigma_reward_model, method='np') for j in range(len(gamma_eval_points))])
        sp.scatter(*np.array(gamma_eval_points).T,c=llhs)
        sp.set_aspect('equal')
        sp.set_title('t = '+str(t_step*i+1))
    plt.gcf().set_figheight(20)
    plt.gcf().set_figwidth(20)
    plt.tight_layout()

In [ ]:
alpha_gt = 45
sigma_reward_gt = 0.1
T = 40

data = helper.generate_data(T, alpha=alpha_gt, sigma_reward=sigma_reward_gt)

plt.scatter(*data['z'].T,c=data['r'])
plt.gca().set_aspect('equal')
plt.colorbar();

In [ ]:
plot_likelihoods(data, sigma_reward_model=0.1,N=2000)

In [ ]:
plot_likelihoods(data, sigma_reward_model=0.3,N=2000)

In [ ]:
plot_likelihoods(data, sigma_reward_model=1.,N=2000)

In [ ]:
alpha_gt = 90
sigma_reward_gt = 0.1
sigma_reward_model = 0.8
T = 60

data = helper.generate_data(T, alpha=alpha_gt, sigma_reward=sigma_reward_gt)
plot_likelihoods(helper.generate_data(T, alpha=alpha_gt, sigma_reward=sigma_reward_gt), sigma_reward_model=sigma_reward_model,N=1000,t_step=5)

In [ ]:
t=3
gamma_eval_points = tfd.Uniform([-.2,-.2],[1.2,1.2]).sample(3000)
llhs = np.array([helper.model_llh(data['z'][:t], data['r'][:t], gamma=gamma_eval_points[j], sigma_reward=sigma_reward_model) for j in range(len(gamma_eval_points))])
plt.scatter(*np.array(gamma_eval_points).T,c=llhs)
plt.gca().set_aspect('equal')
plt.gcf().set_figheight(2.5)
plt.gcf().set_figwidth(2.5)